In [2]:
# 加载库文件
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

#### 1.将输入转化成TFRecord格式并保存



In [3]:
# 定义生成整数类型属性的函数
def _int64_feature(value):
    # 把数据转化成整数list
    int64_list = tf.train.Int64List(value=[value])
    # 把数据转化成整数属性并返回
    return tf.train.Feature(int64_list=int64_list)

# 定义生成字节类型属性的函数
def _bytes_feature(value):
    # 把数据转化成字节类型
    bytes_list = tf.train.BytesList(value=[value])
    # 把数据转化成字节属性并返回
    return tf.train.Feature(bytes_list=bytes_list)

# 定义将样例转化成tf.train.Example格式
def _make_example(pixels, label, image):
    # 将图像数据转化成字符串
    image_raw = image.tostring()
    # 将数据组织成map<string, Feature>的形式
    feature_map = {'pixels':_int64_feature(pixels),
                   'label':_int64_feature(np.argmax(label)),
                   'image_raw':_bytes_feature(image_raw)}
    # 将数据转化成Features
    Features = tf.train.Features(feature=feature_map)
    # 将数据转化成Example
    example = tf.train.Example(features=Features)
    
    # 返回样例数据
    return example

# 定义数据集路径
dataset_path = "../../../../TensorFlow/datasets/MNIST_data/"

# 读取mnist数据集
mnist = input_data.read_data_sets(dataset_path, dtype=tf.uint8, one_hot=True)
train_images = mnist.train.images
train_labels = mnist.train.labels
pixels = train_images.shape[1]
train_num_example = mnist.train.num_examples

# 定义TFRecord文件地址
filename = "../../../../practise/output.tfrecords"

# 输出包含训练数据的TFRecord文件
with tf.python_io.TFRecordWriter(filename) as writer:
    for index in range(train_num_example):
        example = _make_example(pixels, train_labels[index], train_images[index])
        writer.write(example.SerializeToString())
print("TFRecord训练文件已经保存")

# 读取mnist测试数据
test_images = mnist.test.images
test_labels = mnist.test.labels
test_pixels = test_images.shape[1]
test_num_example = mnist.test.num_examples

# 定义tfrecord文件路径
filename = "../../../../practise/output_test.tfrecords"

# 输出包含测试数据的TFRecord文件
with tf.python_io.TFRecordWriter(filename) as writer:
    for index in range(test_num_example):
        example = _make_example(test_pixels, test_labels[index], test_images[index])
        writer.write(example.SerializeToString())
print("TFRecord测试文件已经保存")

Extracting ../../../../TensorFlow/datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../../../TensorFlow/datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../../../TensorFlow/datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../../../TensorFlow/datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
TFRecord训练文件已经保存
TFRecord测试文件已经保存


#### 2.读取TFRecord文件

In [6]:
# 定义训练集tfrecord的路径
train_path = "../../../../practise/output.tfrecords"

# 创建一个reader来读取TFRecord的样例
reader = tf.TFRecordReader()

# 创建一个队列来维护输出文件列表
filename_queue = tf.train.string_input_producer([train_path])

# 从文件中读出一个样例。使用read_up_to函数可以一次性读出多个样例
_, serialized_example = reader.read(filename_queue)

# 解析读取的样例
features = tf.parse_single_example(
    serialized_example,
    features={
        'image_raw':tf.FixedLenFeature([], tf.string),
        'pixels':tf.FixedLenFeature([], tf.int64),
        'label':tf.FixedLenFeature([], tf.int64)
    })

# tf.decode_raw可以将字符串解析成图像对应的像素
images = tf.decode_raw(features['image_raw'], tf.uint8)
labels = tf.cast(features['label'], tf.int32)
pixels = tf.cast(features['pixels'], tf.int32)

sess = tf.Session()

# 启动多线程处理输入数据
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])